Grupo:



*   Carlos Henrique dos Santos
*   Gabriel Scholze Rosa
*   João Vitor Brandão
*   Matheus Leindorf Muller

In [103]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [104]:
# https://www.dropbox.com/s/cjztq44o5k7imwu/transactions_amostra.csv?dl=1

import pyspark
import pyspark.sql.functions
from pyspark.sql import SparkSession
from pyspark import SparkFiles
spark = SparkSession.builder.master('local[*]').appName('Muller').getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

sc = spark.sparkContext
sc.addFile('https://www.dropbox.com/s/cjztq44o5k7imwu/transactions_amostra.csv?dl=1')

In [105]:
df = spark.read.csv("file://" + SparkFiles.get('transactions_amostra.csv'), header=True, inferSchema=True,  sep=";")
df

country_or_area,year,comm_code,commodity,flow,trade_usd,weight_kg,quantity_name,quantity,category
Belgium,2016,920510,Brass-wind instru...,Export,571297,3966.0,Number of items,4135.0,92_musical_instru...
Guatemala,2008,660200,"Walking-sticks, s...",Export,35022,5575.0,Number of items,10089.0,66_umbrellas_walk...
Barbados,2006,220210,"Beverage waters, ...",Re-Export,81058,44458.0,Volume in litres,24113.0,22_beverages_spir...
Tunisia,2016,780411,Lead foil of a th...,Import,4658,121.0,Weight in kilograms,121.0,78_lead_and_artic...
Lithuania,1996,560110,"Sanitary towels, ...",Export,76499,5419.0,Weight in kilograms,5419.0,56_wadding_felt_n...
Denmark,2011,310100,Animal or vegetab...,Export,4903675,1.902844E7,Weight in kilograms,1.902844E7,31_fertilizers
Thailand,1994,920290,String musical in...,Import,2088672,0.0,Number of items,59595.0,92_musical_instru...
Portugal,2004,511119,"Woven fabric, >85...",Export,1546575,87367.0,Weight in kilograms,87367.0,51_wool_animal_ha...
Congo,2011,420690,"Articles of gut, ...",Export,883,9.0,Weight in kilograms,9.0,42_articles_of_le...
Antigua and Barbuda,2016,620332,"Mens, boys jacket...",Export,12988,1403.0,Number of items,648.0,62_articles_of_ap...


In [106]:
from pyspark.sql.functions import col, avg

In [107]:
df.printSchema()

root
 |-- country_or_area: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- comm_code: string (nullable = true)
 |-- commodity: string (nullable = true)
 |-- flow: string (nullable = true)
 |-- trade_usd: long (nullable = true)
 |-- weight_kg: double (nullable = true)
 |-- quantity_name: string (nullable = true)
 |-- quantity: double (nullable = true)
 |-- category: string (nullable = true)



In [108]:
#Question 1
#Filtering the dafaframe with column "country_or_area" value being Brazil as value, and counting occurrences
brazil = df.filter(col("country_or_area").like("Brazil")).count()
brazil

54762

In [109]:
#Question 2
# To get the number of instances by year we group the values by key, then we count the occurrences.
years = df.groupBy("year").count()
years

year,count
1990,18188
2003,101125
2007,105891
2015,101973
2006,109186
2013,104734
1997,77876
1988,8568
1994,54143
2014,103231


In [110]:
#Question 3
# here we group the instances by year and flow type using groupBy(), then we count the occurrences
q3 = df.groupBy("year", "flow").count()
q3

year,flow,count
2011,Import,62270
2006,Import,64345
1994,Re-Export,2094
2009,Export,35489
1998,Import,49870
2012,Export,35241
1999,Re-Export,4049
2006,Export,36981
2015,Re-Import,3005
2014,Re-Import,2902


In [111]:
# Question 4
#grouping by year, we get the average of the trade_usd with avg()
q4 = df.groupBy("year").avg("trade_usd")
q4

year,avg(trade_usd)
1990,1.3192706620958874E7
2003,1.421775659984178E7
2007,2.58920247040825E7
2015,3.18705453775215E7
2006,2.309561335510047E7
2013,4.000417948798862E7
1997,1.0552287985399866E7
1988,1.6750924673436042E7
1994,1.1536598545389062E7
2014,4.07714802568802E7


In [112]:
#Questão 5
#1-filter the dataframe to get only instances with the column flow being Export
#2-filter the dataframe to get only instances with the column country_or_area being Brazil
#3-grouping by year, quantity_name and category
#4-average function avg()
#5-order the result bt average descending
q5 = df.filter(col("flow").like("Export")).filter(col("country_or_area").like("Brazil")).groupBy("year","quantity_name", "category").avg("trade_usd").orderBy(col("avg(trade_usd)").desc())
q5

year,quantity_name,category,avg(trade_usd)
2015,Number of items,88_aircraft_space...,3.031369794E9
2012,Weight in kilograms,27_mineral_fuels_...,2.8609339226666665E9
2001,Number of items,88_aircraft_space...,2.808453981E9
2012,Number of items,88_aircraft_space...,1.790261438E9
2016,Weight in kilograms,26_ores_slag_and_ash,1.66218356E9
2016,Weight in kilograms,17_sugars_and_sug...,1.6572273834E9
2015,Weight in kilograms,23_residues_waste...,1.45606111375E9
2010,Weight in kilograms,27_mineral_fuels_...,1.3656568598333333E9
2014,Weight in kilograms,17_sugars_and_sug...,1.356273969857143E9
2014,Weight in kilograms,09_coffee_tea_mat...,1.2317656416E9


In [113]:
from pyspark.sql import functions
#Questão 6
#1-group by quantity_name and year
#2- agg() to make 3 different functions at the same time: avg(), min() e max, all on the trade_usd column
q6 = df.groupBy('quantity_name','year').agg(functions.avg("trade_usd"),functions.min("trade_usd"),functions.max("trade_usd"))
q6

quantity_name,year,avg(trade_usd),min(trade_usd),max(trade_usd)
Electrical energy...,1994,4122301.5,588,16549948
Number of pairs,2008,3.9991725244176015E7,23,8231914307
Weight in carats,2013,9.718167391666667E7,487,1156063734
Thousands of items,2006,6971520.423076923,58,27188639
Volume in cubic m...,2005,2.1613620009375E7,6,740206429
Area in square me...,2013,1.0891966478880407E7,7,3465872352
Length in metres,1996,1.493895061967213E7,13,558384790
Number of pairs,2013,3.524038769922879E7,15,3087358939
Volume in litres,1988,8.903355118110237E7,6,18423300970
Number of pairs,1990,6225074.855421687,130,119229449


In [114]:
from pyspark.sql import functions as f
#Questao 7
#1-filter to get the only instances with year being 2016
#2-grouping occorrences by flow type and commodity
#3-summing the column quantity by key and renaming the result to sum_quantity
result1 = df.filter(col("year").like("2016")).groupBy("flow","commodity").sum("quantity").withColumnRenamed('sum(quantity)','sum_quantity')
#4-grouping the result1 by flow and getting the max of column sum_quantity, renaming the result to max_quantity
result2 = result1.groupBy("flow").max("sum_quantity").withColumnRenamed("max(sum_quantity)", "max_quantity")
#5-we joing the two results comparing the sum of the quantitys, and select to get only the columns we want
result2.alias("a").join(result1.alias("b"), result1.sum_quantity==result2.max_quantity).select("a.flow","a.max_quantity","b.commodity")

flow,max_quantity,commodity
Import,1.073469712878E12,"Iron ore, concent..."
Export,6.99847369665E11,"Ice, snow and pot..."
Re-Export,1.261968E9,Safety razor blad...
Re-Import,3.8774873E7,"Chem wood pulp, s..."


Grupo:



*   Carlos Henrique dos Santos
*   Gabriel Scholze Rosa
*   João Vitor Brandão
*   Matheus Leindorf Muller